In [ ]:
!pip install transformers 
!pip install datasets

     |████████████████████████████████| 3.8 MB 8.1 MB/s 
     |████████████████████████████████| 6.5 MB 56.2 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 10.0 MB/s 
     |████████████████████████████████| 212 kB 41.6 MB/s 
     |████████████████████████████████| 1.1 MB 54.6 MB/s 
     |████████████████████████████████| 134 kB 54.0 MB/s 
     |████████████████████████████████| 127 kB 72.7 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 73.1 MB/s 
     |████████████████████████████████| 271 kB 66.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Files/cleaned.csv', index_col=0)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

,dialect,0
id,,
1175358310087892992,IQ,لكن بالنهاية .. ينتفض .. يغير .
1175416117793349632,IQ,يعني هذا محسوب على البشر .. حيونه ووحشيه .. وت...
1175450108898565888,IQ,مبين من كلامه خليجي
1175471073770573824,IQ,يسلملي مرورك وروحك الحلوه💐
1175496913145217024,IQ,وين هل الغيبه اخ محمد 🌸🌺


In [ ]:
X = df['0'].to_list()

In [ ]:
Y = pd.get_dummies(df['dialect']).values

In [ ]:
x_big, x_train_test, y_big, y_train_test = train_test_split(X, Y, test_size=.1, stratify=Y, random_state=42)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size=.2, stratify=y_train_test, random_state=42)

In [ ]:
x_train[0]

'أقسى شعور أحن له وغيري شاغله ..'

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(x_train, truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

In [ ]:
import torch

class DilacetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = DilacetDataset(train_encodings, y_train.astype('float64'))
test_dataset = DilacetDataset(test_encodings, y_test.astype('float64'))

In [ ]:
tokenizer.decode(train_dataset[0]["input_ids"])

'[CLS] اقسى شعور احن له وغيري شاغله.. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [ ]:
from transformers import (BertForSequenceClassification, TrainingArguments, Trainer, BertConfig, TextClassificationPipeline)

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = 18
model = BertForSequenceClassification(config) 
model.parameters

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

<bound method Module.parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36625
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6870


Step,Training Loss
10,0.667100
20,0.581700
30,0.456200
40,0.372100
50,0.322600
60,0.297400
70,0.281000
80,0.269000
90,0.255600
100,0.250300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=6870, training_loss=0.20615965841118425, metrics={'train_runtime': 5005.8131, 'train_samples_per_second': 21.949, 'train_steps_per_second': 1.372, 'total_flos': 1.4908513254228e+16, 'train_loss': 0.20615965841118425, 'epoch': 3.0})

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 9157
  Batch size = 64


{'epoch': 3.0,
 'eval_loss': 0.19723298528019018,
 'eval_runtime': 126.1947,
 'eval_samples_per_second': 72.562,
 'eval_steps_per_second': 1.141}

In [ ]:
output=trainer.predict(test_dataset)[0]

***** Running Prediction *****
  Num examples = 9157
  Batch size = 64


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

cm=classification_report(y_test.argmax(axis=1),output2.argmax(axis=1))
print(cm)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       526
           1       0.00      0.00      0.00       525
           2       0.00      0.00      0.00       323
           3       0.27      0.60      0.37      1152
           4       0.00      0.00      0.00       310
           5       0.00      0.00      0.00       558
           6       0.16      0.49      0.24       842
           7       0.22      0.17      0.20       552
           8       0.11      0.14      0.12       729
           9       0.00      0.00      0.00       231
          10       0.00      0.00      0.00       382
          11       0.15      0.46      0.22       875
          12       0.00      0.00      0.00       621
          13       0.00      0.00      0.00       535
          14       0.00      0.00      0.00       289
          15       0.00      0.00      0.00       324
          16       0.00      0.00      0.00       185
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
trainer.save_model('/content/drive/MyDrive/Colab Files/trial2')

Saving model checkpoint to /content/drive/MyDrive/Colab Files/trial2
Configuration saved in /content/drive/MyDrive/Colab Files/trial2/config.json
Model weights saved in /content/drive/MyDrive/Colab Files/trial2/pytorch_model.bin
